<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173/blob/main/Class_03_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------
**COPYRIGHT NOTICE:** This Jupyterlab Notebook is a Derivative work of [Jeff Heaton](https://github.com/jeffheaton) licensed under the Apache License, Version 2.0 (the "License"); You may not use this file except in compliance with the License. You may obtain a copy of the License at

> [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

------------------------

# **BIO 1173: Intro Computational Biology**

**Module 3: Introduction to TensorFlow**

* Instructor: [David Senseman](mailto:David.Senseman@utsa.edu), [Department of Biology, Health and the Environment](https://sciences.utsa.edu/bhe/), [UTSA](https://www.utsa.edu/)


### Module 3 Material

* Part 3.1: Deep Learning and Neural Network Introduction
* Part 3.2: Using Keras to Build Regression Models
* **Part 3.3: Using Keras to Build Classification Models**
* Part 3.4: Saving and Loading a Keras Neural Network
* Part 3.5: Early Stopping in Keras to Prevent Overfitting

## Google CoLab Instructions

You MUST run the following code cell to get credit for this class lesson. By running this code cell, you will map your GDrive to ``` /content/drive``` and print out your Google GMAIL address. Your Instructor will use your GMAIL address to verify the author of this class lesson.

In [ ]:
# You must run this cell first
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    from google.colab import auth
    auth.authenticate_user()
    COLAB = True
    print("Note: Using Google CoLab")
    import requests
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
    print(gcloud_tokeninfo['email'])
except:
    print("**WARNING**: Your GMAIL address was **not** printed in the output below.")
    print("**WARNING**: You will NOT receive credit for this lesson.")
    COLAB = False

Mounted at /content/drive
Note: Using Google CoLab
david.senseman@gmail.com


Make sure your GMAIL address is included as the last line in the output above.

# **Neural Network Classification and Regression**

![Neural Network Classification and Regression](https://biologicslab.co/BIO1173/images/class_2_ann_class_reg.png "Neural Network Classification and Regression")

## **Using Keras to Build Classification Models**

In the previous lesson (Class_03_2) we looked at how to use Keras/Tensorflow to build simple neural network models that performed a regression analysis. In this lesson, we continue our introduction into Keras and Tensorflow by building neural network models that perform a **_classification_**.

**_Classification_** is how a neural network attempts to classify the input into one or more **_classes_**.  The simplest way of evaluating a classification network is to track the percentage of training set items classified incorrectly.  

We typically score human results in this manner.  For example, you might have taken multiple-choice exams in school in which you had to shade in a bubble for choices A, B, C, or D.  If you chose the wrong letter on a 10-question exam, you would earn a 90%.  In the same way, we can grade computers; however, most classification algorithms do not merely choose A, B, C, or D.  Computers typically report a classification as their percent confidence in each class.  The figure below shows how a computer and a human might respond to question number 1 on an exam.

**Classification Neural Network Output**
![Classification Neural Network Output](https://biologicslab.co/BIO1173/images/class-multi-choice.png)

As you can see, the human test taker marked the first question as "B." However, the computer test taker had an 80% (0.8) confidence in "B" and was also somewhat sure with 10% (0.1) on "A." The computer then distributed the remaining points to the other two.  

In the simplest sense, the machine would get 80% of the score for this question if the correct answer were "B." The computer would get only 5% (0.05) of the points if the correct answer were "D."

We previously saw how to train a neural network to predict either the `Ripeness` or the `Acidity` of an apple. In this lesson, we will now see how to build a neural network to predict a **_class_**. In particular, we are going to build a neural network that can predict the **_species_** of a particular Iris flower based either on it's sepal dimensions or on its petal dimensions.

The code to construct a neural network that can classify Iris flowers is similar to the code for building a neural network for regression. However, there are 3 important differences in classification models:

* The output neuron count is **not 1**, but matches the number of classes.
* The **_Softmax_** transfer function is utilized by the output layer.
* The loss function is **_cross entropy_**.
  

## Example 1: Predict Iris Species

In Example 1, our goal will be to predict the `class` (species) of an _Iris_ flower based _only_ on the length and width of its **_sepals_**. In **Example 1** you will repeat the analysis, but your model will only use the **_petal_** measurements for training. Although sepals and petals in Iris flowers look similar, they are distinctly different if you look closely.

The next image shows an example of how sepal and petal measurements are made.

**Iris Sepal and Petal Measurements**

![____](https://biologicslab.co/BIO1173/images/iris_petal_sepal.png)

As was done in Lesson_03_3, the code for Example 1 will be divided into the following 3 steps:

1. Read datafile and create DataFrame
2. Create feature vector
3. Construct, compile and train the neural network model

When constructing neural networks, AI researchers typically follow the same 3 steps.

### Example 1-Step 1: Read datafile and create DataFrame

The first step is usually to either read the a dataset from a local file, or download one from the Internet. In this example, we will read the Iris Flower dataset stored in a CSV file called `iris.csv` on the course HTTPS server, `https://biologicslab.co`, using this code chunk:
~~~text
# Read dataset into a DataFrame
sepalDF = pd.read_csv(
    "http://biologicslab.co/BIO1173/data/iris.csv",
            na_values=['NA', '?'])
~~~
As the dataset is being read, it is being stored in a Pandas DataFrame called `sepalDF`. This name was choosen to remind us that this DataFrame is being used for Example 1. In this example, we are going to **_shuffle_** and reindex the data after setting the random seed = 42.

Finally, after creating our new DataFrame and shuffling the data, we display the first few rows and columns to make sure the data was read correctly.

In [ ]:
# Example 1: Read datafile and create DataFrame

import pandas as pd
import numpy as np

# Read dataset into a DataFrame
sepalDF = pd.read_csv(
    "http://biologicslab.co/BIO1173/data/iris.csv",
            na_values=['NA', '?'])

# Set the random seed to 42
np.random.seed(42)

# Shuffle & reindex
sepalDF = sepalDF.reindex(np.random.permutation(sepalDF.index))

# Set the max rows and max columns
pd.set_option('display.max_rows', 8)
pd.set_option('display.max_columns', 8)

# Display the DataFrame
display(sepalDF)


If your code is correct, you should see the following table:

![___](https://biologicslab.co/BIO1173/images/class_03/class_03_3_image01a.png)


As you can see, all of the information in the DataFrame is numeric, with the exception of the column `species`. We need to keep this mind when we create our feature vector in the next step.

### Example 1-Step 2: Create Feature Vector

While we normally begin by converting all the string values to integers, we are **_not_** going to do this in this particular example. The only column with non-numeric values is the column `species`. Since the `species` column will be our Y-values, we take care of it **_after_** we first generate our X-values.

Normally we would like to use **all** of the useful data that is available in a dataset for training our neural network. However, since this is a teaching example, we will limit our X-values to just the sepal measurements. We can limit what columns in our DataFrame are used for generating the X-values by simply identifying the specific columns that we want to use by name, as shown in the following code chunk:
~~~text
# Generate X-values
sepalX = sepalDF[['sepal_length', 'sepal_width']].values
sepalX = np.asarray(sepalX).astype('float32')
~~~
When generate either X or Y values, we always want to make sure that they are type `float32` by using the code line:
~~~text
sepalX = np.asarray(sepalX).astype('float32')
~~~

One **critical difference** between classification neural networks and regression neural networks is how the Y-values are generated. In regression, you can directly use the numerical values in the correct column in the DataFrame as was demonstrated in the previous lesson. However, when creating a feature vector for a classification neural network, **_the Y-values must be One-Hot Encoded!_**

Here is the code chunk for generating the Y-values for classification using One-Hot Encoding:
~~~text
# Generate Y-values
dummies = pd.get_dummies(sepalDF['species'], dtype=int) # Classification
iris_species = dummies.columns
sepalY = dummies.values
sepalY = np.asarray(sepalY).astype('float32')
~~~

To verify that our coding was correct, the example prints the first 10 values in the variable `dummies` and our Numpy array containing the Y-values, `sepalY`.


In [ ]:
# Example 1-Step 2: Create Feature Vector

import pandas as pd
import numpy as np

# Generate X-values
sepalX = sepalDF[['sepal_length', 'sepal_width']].values
sepalX = np.asarray(sepalX).astype('float32')

# Generate Y-values
dummies = pd.get_dummies(sepalDF['species'], dtype=int) # Classification
iris_species = dummies.columns
sepalY = dummies.values
sepalY = np.asarray(sepalY).astype('float32')

# Print dummies
print(dummies[0:10])

# Print first 10 Y-values
print(sepalY[0:10])


If the code is correct, you should see the following table:

![___](https://biologicslab.co/BIO1173/images/class_03/class_03_3_image04a.png)

The variable called `dummies` is actually a DataFrame that was generated as part of the One-Hot Encoding. The first 10 records in the `dummies` DataFrame are shown at the top of this output. You should note that the `dummies` DataFrame has **exactly** the same numeric values as the Numpy array `sepalY` shown at the bottom of the output. This makes sense given that `sepalY` was generated by the following code chunk:
~~~text
sepalY = dummies.values
~~~
This line of code simply converts the numeric **_values_** in the `dummies` DataFrame into a Numpy array, `sepalY`.

For example, the first flower in `dummies` (index number `73`) has a `1` in the second column `Iris-versicolor`. If you look at the first element in the `sepalY` array, there is also a `1` in the second column.

### Example 1-Step 3: Construct, Compile and Train Neural Network

As mentioned above, the neural network that we will use to classify Iris flowers based on their sepal dimensions, is very similar to a regression neural network but with the following exceptions:

* The output neuron count matches the number of classes (in this case 3, since there are 3 different species).
* The `Softmax` transfer function is utilized by the output layer.
* The loss function is `cross entropy`.

The code in the cell below builds a simple neural network called `sepalModel`. Notice that the number of neurons in the input layer is specified by `input_dim=sepalX.shape[1]`:
~~~text
sepalModel.add(Dense(50, input_dim=sepalX.shape[1],
                     activation='relu')) # Hidden 1
~~~
Also the output layer of this classification neural network is different:
~~~text
sepalModel.add(Dense(sepalY.shape[1],activation='softmax')) # Output
~~~
First, the are more than 1 neuron in the output layer. In a classification neural network, there must be 1 neuron for each class. Since there are 3 classes (species) in the Iris Flower dataset, there has to be 3 output neurons. The number of output neurons is specified by the argument `sepalY.shape[1]`.

The other difference in the output layer is the presence of an activation function. In this example, the activation function is `softmax`. In a regression neural network, the single output neuron does **not** have an activation function.

Finally, you should notice that when the model is compiled, it uses the `categorical_crossentropy` loss function instead of the `mean_squared_error` loss function used in regression neural networks.

In [ ]:
# Example 1-Step 2: Construct, compile and train neural network

import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Input
from keras.optimizers import Adam

# Construct model
sepalModel = Sequential()
sepalModel.add(Input(shape=(sepalX.shape[1],))) # Hidden 1
sepalModel.add(Dense(25, activation='relu')) # Hidden 2
sepalModel.add(Dense(sepalY.shape[1],activation='softmax')) # Output

# Compile the model
sepalModel.compile(loss='categorical_crossentropy', optimizer='adam')

# Print model summary (optional)
sepalModel.summary()

# Train model
sepalModel.fit(sepalX,sepalY,verbose=2,epochs=100)

If your code is correct you should see the following output:

~~~text
Model: "sequential_2"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 25)                  │              75 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │              78 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘
 Total params: 153 (612.00 B)
 Trainable params: 153 (612.00 B)
 Non-trainable params: 0 (0.00 B)
Epoch 1/100
5/5 - 1s - 182ms/step - loss: 2.2706
Epoch 2/100
5/5 - 0s - 7ms/step - loss: 2.1261
Epoch 3/100
5/5 - 0s - 7ms/step - loss: 2.0115
Epoch 4/100
5/5 - 0s - 7ms/step - loss: 1.9095
Epoch 5/100
5/5 - 0s - 6ms/step - loss: 1.8233p
..................

Epoch 95/100
5/5 - 0s - 11ms/step - loss: 0.8004
Epoch 96/100
5/5 - 0s - 6ms/step - loss: 0.7969
Epoch 97/100
5/5 - 0s - 6ms/step - loss: 0.7937
Epoch 98/100
5/5 - 0s - 6ms/step - loss: 0.7903
Epoch 99/100
5/5 - 0s - 6ms/step - loss: 0.7877
Epoch 100/100
5/5 - 0s - 11ms/step - loss: 0.7844
<keras.src.callbacks.history.History at 0x79bfedba82e0>
~~~

### Example 2: Print Predictions

Now that we have trained our neural network, `sepalModel`, we would like to be able to use it. As before, we will generate predictions. Instead of given us a single prediction, as the regression neural network model did in Class_03_2, our new model will make **_3 predictions_**, one prediction for each **_class_** in the dependent variable. These three predictions represents the 3 probabilities that a particular unknown flower is (1) _Iris setosa_, (2) _Iris versicolor_, and (3) _Iris virginica_.

In [ ]:
# Example 2: Print Predictions

import numpy as np

# Compute the model predictions
sepalPred = sepalModel.predict(sepalX)

# Change print from scientific notation
np.set_printoptions(suppress=True)

# Print out the results
print(f"Shape of sepalPred: {sepalPred.shape}")
print(sepalPred[0:10])

If your code is correct your should see something similar to the following:

~~~text
5/5 [==============================] - 0s 2ms/step
Shape of sepalPred: (150, 3)
[[0.00667937 0.49824646 0.49507418]
 [0.9701563  0.02258347 0.00726027]
 [0.00000133 0.2490988  0.75089985]
 [0.02025999 0.5275467  0.4521932 ]
 [0.00030747 0.40100244 0.59869015]
 [0.8894263  0.07975859 0.0308151 ]
 [0.10499485 0.5329848  0.3620204 ]
 [0.00158804 0.4518695  0.5465425 ]
 [0.0000668  0.3567878  0.6431454 ]
 [0.01239389 0.51633203 0.47127402]]
~~~

Each row of numbers represents the model's prediction for one Iris flower in the dataset. The first column represents the **_probability_** that the flower's species is _I. setosa_, the second column is the probability for _I. versicolor_, and the third column is the probability for _I. virginica_.

You should note two things about these predictions. First, generally, one column has a significantly higher probability than the other two columns, but this is not always the case.

In the particular example shown above, there is a very low probability that the first flower is _I. setosa_ (prob=0.0067), but the `sepalModel` assigned essentially equal probabilites to the flowere being  _I. versicolor_ (prob=0.498 or 49.8%) and _I. virginica_ (prob=0.495 or 49.5%). In other words, our `sepalModel` thinks there is a 50=50 chance that this particular flower is either  _I. versicolor_ or _I. virginica_.

For the next flower on the list, our `sepalModel` is very confident (prob=0.970 or 97%) that it is  _I. setosa_. Finally, `sepalModel` is moderately confident (prob=0.750 or 75%) that the third flower is _I. virginica_ but, the is some possibilty (prob=0.249 or 25%) that it is really _I. virginica_.  

As you might expect, when you add up all three probabilities for any particular flower, the sum is essentially zero. To demonstrate this, the next code cell adds the three probabilites of the last flower in the list.

In [ ]:
# Add up the probabilites for list flower in the list
0.01239389 + 0.51633203 + 0.47127402

As you can see, it's pretty close to 1. That makes sense since there is a 100% probability that any flower will be one of the three possible species in the dataset.

### Example 3: Print Predicted and Expected Values

Usually, the program considers the column with the **_highest_** prediction to be the prediction of the neural network. The `np.argmax()` function can be used to find the index of the **_maximum prediction_** for each row.

As shown in the cell below, we can use `np.argmax()` function generate the variables `sepalPredict_classes` and `sepalExpected_classes`. We will need these variables shortly when we want to compute the model's accuracy score.


In [ ]:
# Example 3: Print Predicted and Expected Values

import numpy as np

# Find the maximum prediction for each row
sepalPredict_classes = np.argmax(sepalPred,axis=1)

# Find the expected value for each row
sepalExpected_classes = np.argmax(sepalY,axis=1)

# Print out the results
print(f"Predictions: {sepalPredict_classes}")
print(f"Expected: {sepalExpected_classes}")

If your code is correct you should see something similar to the following:

~~~text
Predictions: [2 0 2 1 2 0 1 2 2 1 2 0 0 0 0 1 2 2 1 2 0 1 0 2 2 2 2 2 0 0 0 0 2 0 0 2 1
 0 0 0 1 1 2 0 0 2 2 2 2 2 1 2 1 0 2 1 0 0 0 2 2 0 0 0 1 0 1 2 0 2 2 0 1 1
 2 1 1 2 0 1 2 0 0 2 2 0 1 0 0 1 1 2 2 2 2 1 0 0 2 2 0 0 0 2 1 0 2 1 0 2 2
 2 2 1 0 2 1 2 2 1 1 1 2 1 0 1 2 2 0 1 2 2 0 2 0 2 2 2 1 2 2 2 1 1 0 2 1 0
 2 2]
Expected: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0 1 2 2 1 2 1 2 1 0 2 1 0 0 0 1 2 0 0 0 1 0 1 2 0 1 2 0 2 2
 1 1 2 1 0 1 2 0 0 1 1 0 2 0 0 1 1 2 1 2 2 1 0 0 2 2 0 0 0 1 2 0 2 2 0 1 1
 2 1 2 0 2 1 2 1 1 1 0 1 1 0 1 2 2 0 1 2 2 0 2 0 1 2 2 1 2 1 1 2 2 0 1 2 0
 1 2]
~~~

### Example 4 - Convert Index Values into Species Names

With a little bit of python coding, it's not too hard to change the numbers `0`, `1` and `2` into their corresponding species names. The trick is the variable `iris_species` which is Python type called `index`. This variable was created as part of the One-Hot Encoding of the `species` column during the creation of the feature vector in Example 1-Step 2.

If you print out `iris_species` you get the following:
~~~text
Index(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype='object')
~~~~
The code in the next cell shows how you can use this `index` type to convert index values in the variable `sepalPred` to their species names.

In [ ]:
# Example 4: Convert index values into species names

# Print index values for first 5 flowers
print(f"sepalPredict_classes: {sepalPredict_classes[0:4]}")

# Convert index values to species names
print(*iris_species[sepalPredict_classes[0:4]])

If your code is correct you should see something _similar_ to:
~~~text
sepalPredict_classes: [1 0 2 1]
Iris-versicolor Iris-setosa Iris-virginica Iris-versicolor
~~~
As you can see, the code in the cell above converted the 4 index values `[1 0 2 1]` into the 4 species names `Iris-versicolor`,`Iris-setosa`, `Iris-virginica`, and `Iris-versicolor`.

### Example 5: Compute the Accuracy Score

When using a regression neural network, the convention is to use Root Mean Squared Error (RMSE) as a meaure of the model's ability to correctly predict the Y-values (see Class_03_2). As we dicussed previously, interpreting how good a model is by it's RSME is tricky.

The situation is mucn better when working with classification neural networks. Instead of measuring performance with RSME, we use an **_Accuracy Score_** which is a more easily understood error metric. An accuracy score is essentially a test score. As an example, for all of the species predictions made by the `sepalModel`, what percent were correct? In other words, the accuracy score is like the number of correct answers on an exam.  

Unfortunatley, the accuracy score isn't a perfect error metric because it doesn't consider how confident the neural network was in each prediction.

The code in the cell below uses the `accuracy_score()` function imported from the **scikit-learn** package (nickname `sklearn`), to compute the accuracy of the predicitions made by `sepalModel` and stores this value in a variable called `sepalCorrect`.

In [ ]:
# Example 5: Compute the accuracy score

from sklearn.metrics import accuracy_score

# Compute accuracy
sepalCorrect = accuracy_score(sepalExpected_classes,sepalPredict_classes)

# Print out the results
print(f"Accuracy: {sepalCorrect}")

If your code is correct, you should see something similar to the following output:
~~~text
Accuracy: 0.6266666666666667
~~~
The accuracy of the `sepalModel` is roughly 60% accurate.

### Example 6: Use Model to Make an _ad hoc_ Prediction

The code below performs an _ad hoc_ prediction. Suppose we measure the sepal length and width of flower from an unknown Iris species. We can "feed" this information into our trained neural network model `sepalModel` and ask it to predict which species did the flower came from. This is an example of an _ad hoc_ prediction.

The first step is to define a variable `sample_flower` with the measurements (in cm) for the sepal length (6.6) and the sepal width (2.9):
~~~text
sample_flower = np.array( [[6.6,2.9]], dtype=float)
~~~

Since the model `sepalModel` was trained on data where the sepal length was "first" and sepal width was "second", it is essential to submit the sepal measurements in exactly the same order.

The next line of code uses the `sepalModel` to make the actual prediction which is stored in the variable `sepalPred`:
~~~text
sepalPred = sepalModel.predict(sample_flower)
~~~

In [ ]:
# Example 6: Use model to make an ad hoc prediction

# Specify the sepal length and width for an unknown Iris flower
sample_flower = np.array( [[6.6,2.9]], dtype=float)

# Use the neural network to predict the species
sepalPred = sepalModel.predict(sample_flower)

# Print out the results
print(sepalPred)
sepalPred = np.argmax(sepalPred)
print(f"Model predicts sepal dimensions {sample_flower} are from: {iris_species[sepalPred]}")

If your code is correct you should see something similar to the following output:
~~~text
1/1 [==============================] - 0s 20ms/step
[[0.0014901  0.45076337 0.5477465 ]]
Model predicts sepal dimensions [[6.6 2.9]] are from: Iris-virginica
~~~
While our `sepalModel` predicted that the unknown flower was Iris-virginica, by looking at the output, we can see that it isn't very confident with this prediction. Look at the line:
~~~text
[[0.0014901  0.45076337 0.5477465 ]]
~~~
These three numbers are what the model thinks are the probabilites that the flower is Iris-setosa, Iris-versicolor _or_ Iris-virginica, respectively.

Specifically, the model predicted there was a very small probabilty (0.0014901) that the sample flower species was Iris-setosa. However, it had considerable trouble deciding if the flower species was Iris-versicolor (0.45076337 or 45% chance) or Iris-virginica (0.5477465 or 55% chance). Going with the highest probability, the model "guessed" that it was Iris-virginica, but it still recorded that there was a 45% chance that real species was Iris-versicolor.   

### Example 7: Use Model to Make Two _ad hoc_ Predictions

You can also predict two sample flowers by stacking the sepal mesurements.
~~~text
# Specify the sepal length and width for the two Iris flower
sample_flowers = np.array( [[5.9,2.8],[5.1,3.5]],\
        dtype=float)
~~~
The code for making the actual predictions is the same:
~~~text
sepalPred = sepalModel.predict(sample_flowers)
~~~
Notice that the **argmax** in the second prediction requires **axis=1**.  Since we have a 2D array now, we must specify which axis to take the **argmax** over.  The value **axis=1** specifies we want the max column index for each row.
~~~text
sepalPred = np.argmax(sepalPred, axis=1)
~~~

In [ ]:
# Example 7: Use the model to make two ad hoc predictions

# Specify the sepal length and width for the two Iris flower
sample_flowers = np.array( [[5.9,2.8],[5.1,3.5]],\
        dtype=float)

# Use the neural network to predict the species
sepalPred = sepalModel.predict(sample_flowers)

# Print out the results
print(sepalPred)
sepalPred = np.argmax(sepalPred, axis=1)
print(f"Model predicts sepal dimensions {sample_flowers} are from: {iris_species[sepalPred]}")


If your code is correct you should see something similiar to the following output:
~~~text
1/1 [==============================] - 0s 21ms/step
[[0.0158558  0.5222274  0.46191677]
 [0.9817724  0.0140365  0.00419102]]
Model predicts sepal dimensions [[5.9 2.8]
 [5.1 3.5]] are from: Index(['Iris-versicolor', 'Iris-setosa'], dtype='object')
~~~

Looking at the output shown above, the `sepalModel` again had problems deciding between Iris-versicolor (prob=0.5222274 or 52%) and Iris-virginica (prob=0.46191677 or 46%) for the first flower, but was quite certain (prob=0.9817724 or 98%) that the second flower was from an Iris-setosa plant.

## **Exercise 1: Predict Iris Species**

For **Exercise 1** you are build the same neural network that was created in Example 1. Call your new neural network `petalModel`. The only real difference with your model will be the _features_ used to predict the species of an Iris flower. Rather than use sepal dimensions, you will train your model using **_petal_** dimensions (i.e. petal length and petal width).

### **Exercise 1-Step 1: Read datafile and create DataFrame**

In the cell below, write the code to read the CVS file, `iris.csv` from the course HTTPS server and store the information in a new DataFrame called `petalDF`. Shuffle and reindex the data, making sure to change the value of the random seed. Set your new `random seed = 30`.

**WARNING: If you don't change the value of the random seed to `30` you won't earn full credit for this lesson!**

After shuffling the data,  display the first 8 rows and columns to make sure the data was read correctly and your DataFrame was shuffled using the correct random seed.

In [ ]:
# Insert your code for Exercise 1-Step 1 here




If your code is correct, you should see the following table:

![___](https://biologicslab.co/BIO1173/images/class_03/class_03_3_image02a.png)

If you table looks similar but has different numbers, it means you forgot to set the random seed = 30.

### **Exercise 1-Step 2: Create Feature Vector**

In the cell below create a feature vector for your neural network model.

You will need to modify the code used to generate your X-values. Call your X-values `petalX` and only use the data in the columns called `petal_length` and `petal_width`.

Since your feature vector will be used for classification, you will need to One-Hot Encode the column `species`. The code for doing this is exactly the same as the code in Example 1-Step 2 **_except_** you must call the array holding your Y-values `petalY`.

As always, make sure both your X-values and Y-values are in the correct, `float32` format.

Finally, print out the first 10 values in both the DataFrame `dummies` and your `petalY` array.

In [ ]:
# Insert your code for Exercise 1-Step 2 here



If your code is correct, you should see something similar to this table:

![___](https://biologicslab.co/BIO1173/images/class_03/class_03_3_image03a.png)


### **Exercise 1-Step 3: Construct, Compile and Train Neural Network**

In the cell below, write the code to build a neural network called `petalModel`. Since your new model will trained on different data, training will cause your model to have significantly different weights and biases between neurons.  

Make sure to set the input dimension to `petalX` as shown in this code chunk:
~~~text
petalModel.add(Input(shape=(petalX.shape[1],)))  # Input layer
~~~
Similarily, sure to set the output dimension to `petalY` as shown in this code chunk:
~~~text
petalModel.add(Dense(petalY.shape[1],activation='softmax')) # Output
~~~
As mentioned previously, when students in the course create neural network using "copy-and-paste", the often forget to change these values.

Finally, don't forget that you need to train your `petalModel` using the X-values in `petalX` and the Y-values in `petalY`.

In [ ]:
# Insert your code for Exercise 1-Step 3 here



If your code is correct you should see something similar to the following output:

~~~text
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 dense_5 (Dense)             (None, 50)                150       
                                                                 
 dense_6 (Dense)             (None, 25)                1275      
                                                                 
 dense_7 (Dense)             (None, 3)                 78        
                                                                 
=================================================================
Total params: 1,503
Trainable params: 1,503
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
5/5 - 0s - loss: 1.0804 - 265ms/epoch - 53ms/step
Epoch 2/100
5/5 - 0s - loss: 1.0275 - 28ms/epoch - 6ms/step
Epoch 3/100
5/5 - 0s - loss: 1.0020 - 37ms/epoch - 7ms/step
Epoch 4/100
5/5 - 0s - loss: 0.9877 - 27ms/epoch - 5ms/step
Epoch 5/100
5/5 - 0s - loss: 0.9729 - 27ms/epoch - 5ms/step

.......................

Epoch 95/100
5/5 - 0s - loss: 0.1660 - 18ms/epoch - 4ms/step
Epoch 96/100
5/5 - 0s - loss: 0.1647 - 19ms/epoch - 4ms/step
Epoch 97/100
5/5 - 0s - loss: 0.1629 - 20ms/epoch - 4ms/step
Epoch 98/100
5/5 - 0s - loss: 0.1606 - 19ms/epoch - 4ms/step
Epoch 99/100
5/5 - 0s - loss: 0.1629 - 20ms/epoch - 4ms/step
Epoch 100/100
5/5 - 0s - loss: 0.1607 - 19ms/epoch - 4ms/step

<keras.callbacks.History at 0x27e47fa9610>


### **Exercise 2: Print Predictions**

In the cell below compute the predictions made by your `petalModel` and store these values in a new variable called `petalPred`. Print out the predictions for the first 10 Iris flowers.

In [ ]:
# Insert your code for Exercise 2 here



If your code is correct you should see something similar to the following:

~~~text
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Shape of petalPred: (150, 3)
[[0.23585553 0.35244355 0.41170087]
 [0.23972444 0.34498298 0.41529265]
 [0.24676624 0.3501725  0.40306118]
 [0.11367034 0.309354   0.57697564]
 [0.1497996  0.32740334 0.5227971 ]
 [0.15245931 0.33088863 0.516652  ]
 [0.11925902 0.3123483  0.56839263]
 [0.11202369 0.3069624  0.581014  ]
 [0.13606915 0.32362908 0.54030174]
 [0.0819635  0.297499   0.6205375 ]]
~~~

### **Exercise 3:  Print Predicted and Expected Values**

In the cell below, print the predicted and the actual values (class) for your `petalModel`.

To do this you need to find the maximum prediction for each row using this code chunk:
~~~text
# Find the maximum prediction for each row
petalPredict_classes = np.argmax(petalPred,axis=1)
~~~
You will also have to find the expected value for each row using this code chunk:
~~~text
# Find the expected value for each row
petalExpected_classes = np.argmax(petalY,axis=1)
~~~

Once you have determined these values, you can print the results with this code chunk:
~~~text
# Print out the results
print(f"Predictions: {petalPredict_classes}")
print(f"Expected: {petalExpected_classes}")
~~~~

In [ ]:
# Insert your code for Exercise 3 here



If your code is correct you should see something similar to the following output:

~~~text
Predictions: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
Expected: [0 0 0 2 1 1 2 2 1 2 0 2 1 1 0 1 0 0 0 1 2 0 0 0 2 2 1 2 0 1 2 1 2 2 2 2 1
 2 1 2 2 2 0 1 2 1 1 1 1 1 0 1 2 1 0 1 0 1 1 1 0 1 0 1 0 2 2 2 0 2 1 2 0 0
 1 0 2 2 2 2 0 1 0 1 1 1 2 0 1 0 1 2 1 0 0 0 2 2 0 1 1 1 0 0 0 0 2 0 2 0 0
 2 2 2 0 2 1 1 2 1 1 1 1 2 0 1 0 1 0 0 0 2 1 0 0 1 2 2 2 2 0 1 2 0 2 1 2 2
 0 0]
 ~~~

This output shows your model's prediction of each flower's species (top array) as well as their actual (expected) species name (bottom array). The number `0` means Iris-setosa, `1` means Iris-versicolor and `2` means Iris-virginica. Just by a simple, visual comparison, you can see that your model did a very good job predicting a flower's species based on its petal dimensions.


### **Exercise 4: Convert Index Values into Species Names**

In the cell below, print out the first 5 values in `petalPredict_classes` and then using the index variable `iris_species`, print the species names of these first 5 flowers.


In [ ]:
# Insert your code for Exercise 4 here



If your code is correct you should see something similar to the following output:
~~~text
petalPredict_classes: [2 2 2 2]
Iris-virginica Iris-virginica Iris-virginica Iris-virginica
~~~


### **Exercise 5: Compute the Accuracy Score**

In the cell below, compute the accuracy score for your `petalModel` neural network model. Use the f-sting print statement to print the accuracy score.


In [ ]:
# Insert your code for Exericse 5 here



If your code is correct, you should see the following output:
~~~text
Accuracy: 0.3333333333333333
~~~

Not a very good accuracy score. It turns out that there are other statistical procedures such _K_ -Fold Classification that are much more accurate than neural networks for this kind of data.

This accuracy for your `petalModel` is substantially higher than the accuracy of `sepalModel` created in Example 1.

Since the same neural network architecture was used for both models, this difference in accuracy must mean that **_petal_** dimensions are more different in these 3 Iris species than their **_sepal_** dimensions.

You can verify this fact by comparing the mean dimensions for sepal and petals shown below in the Appendix.

### **Exercise 6: Use the Model to Make an _ad hoc_ Prediction**

Use your `petalModel` to make an _ad hoc_ prediction of an unknown Iris species. You measured the petal length to be 1.5 cm and a petal width of 0.3 cm. Print out the model's prediction.

In [ ]:
# Insert your code for Exercise 6 here



If your code is correct you should see something similar to the following:
~~~text
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[[0.24877104 0.3464312  0.40479776]]
Model predicts petal dimensions [[1.5 0.3]] are from: Iris-virginica
~~~


### **Exercise 7: Use Model to Make Two _ad hoc_ Predictions**

Use your `petalModel` to make 2 _ad hoc_ predictions. The petal dimensions are 1.5 and 0.25 cm for the first flower and 5.6 and 2.0 cm for the second flower. As in Exercise 6, print out the model's predictions as probabilies and then the two highest probabilites as the model's final choices.

In [ ]:
# Insert your code for Exercise 7 here



If your code is correct you should see something similar to the following:
~~~text
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
[[0.24776861 0.34829962 0.40393177]
 [0.10021927 0.30179378 0.59798694]]
Model predicts petal dimensions [[1.5  0.25]
 [5.6  2.  ]] are from: Index(['Iris-virginica', 'Iris-virginica'], dtype='object')
~~~

## **Lesson Turn-in**

When you have completed and run all of the code cells, use the **File --> Print.. --> Save to PDF** to generate a PDF of your Colab notebook. Save your PDF as `Class_03_3.lastname.pdf` where _lastname_ is your last name, and upload the file to Canvas.

---------------------------------------------

### **Appendix**

One of the more interesting results of this lesson was the ability of a very simple neural network to more accurately predict a flower's species when it had been trained on petal dimensions compared to sepal dimensions. Specifically, the accuracy of your `petalModel` was close to 95% while the `sepalModel` was only about 75% accurate.

Since these two neural networks had exactly the same architecture, and were both trained for 100 epochs, the difference in accuracy must be attributable to species specific differences in sepal and petal dimensions. In other words, the **_range_** in petal dimensions must be more pronounced in these 3 Iris species than their sepal dimensions.  

In the cells below, the Pandas `df.groupby()` function was used to compute the mean values of sepal length, sepal width, petal width, and petal width in the three different Iris flower species, _I. setosa_, _I. versicolor_, and _I. virginica_.

As expected, species differences for the mean petal dimensions are much greater than the mean sepal dimensions. These differences are sufficient to explain why the accuracy of your `petalModel` was so much higher (\~95% accurate) than the `sepalModel` (\~75% accurate).


In [ ]:
# Group Iris flower species by mean Sepal Length
print("Mean SEPAL LENGTHS by species")
sepal_length = petalDF.groupby('species')['sepal_length'].mean()
sepal_length

In [ ]:
# Group Iris flower species by mean Sepal Width
print("Mean SEPAL WIDTHS by species")
sepal_width = petalDF.groupby('species')['sepal_width'].mean()
sepal_width

In [ ]:
# Group Iris flower species by mean Petal Length
print("Mean PETAL LENGTHS by species")
petal_length = petalDF.groupby('species')['petal_length'].mean()
petal_length

In [ ]:
# Group Iris flower species by mean Petal Width
print("Mean PETAL WIDTHS by species")
petal_width = petalDF.groupby('species')['petal_width'].mean()
petal_width

## **Poly-A Tail**


## **Women in Computing**


![__](https://upload.wikimedia.org/wikipedia/commons/a/ac/Marissa_Mayer_LeWeb_2008_VI.jpg)

#### **Former vice-president of Google Search Products and User Experience, former president and CEO of Yahoo!, Marissa Mayer**

"Tedious" computing and calculating was seen as "women's work" through the 1940s resulting in the term "kilogirl", invented by a member of the Applied Mathematics Panel in the early 1940s. A kilogirl of energy was "equivalent to roughly a thousand hours of computing labor." While women's contributions to the United States war effort during World War II was championed in the media, their roles and the work they did was minimized. This included minimizing the complexity, skill and knowledge needed to work on computers or work as human computers.[46] During WWII, women did most of the ballistics computing, seen by male engineers as being below their level of expertise. Black women computers worked as hard (or more often, even harder) as their white counterparts, but in segregated situations. By 1943, almost all people employed as computers were women; one report said "programming requires lots of patience, persistence and a capacity for detail and those are traits that many girls have".

NACA expanded its pool of women human computers in the 1940s. NACA recognized in 1942 that "the engineers admit themselves that the girl computers do the work more rapidly and accurately than they could." In 1943 two groups, segregated by race, worked on the east and west side of Langley Air Force Base.[51] The black women were the West Area Computers. Unlike their white counterparts, the black women were asked by NACA to re-do college courses they had already passed and many never received promotions.

Women were also working on ballistic missile calculations. In 1948, women such as Barbara Paulson were working on the WAC Corporal, determining trajectories the missiles would take after launch.

Women worked with cryptography and, after some initial resistance, many operated and worked on the Bombe machines. Joyce Aylard operated the Bombe machine testing different methods to break the Enigma code. Joan Clarke was a cryptographer who worked with her friend, Alan Turing, on the Enigma machine at Bletchley Park. When she was promoted to a higher salary grade, there were no positions in the civil service for a "senior female cryptanalyst," and she was listed as a linguist instead. While Clarke developed a method of increasing the speed of double-encrypted messages, unlike many of the men, her decryption technique was not named after her. Other cryptographers at Bletchley included Margaret Rock, Mavis Lever (later Batey), Ruth Briggs and Kerry Howard. In 1941, Batey's work enabled the Allies to break the Italians' naval code before the Battle of Cape Matapan. In the United States, several faster Bombe machines were created. Women, like Louise Pearsall, were recruited from the WAVES to work on code breaking and operate the American Bombe machines.

Hedy Lamarr and co-inventor, George Antheil, worked on a frequency hopping method to help the Navy control torpedoes remotely. The Navy passed on their idea, but Lamarr and Antheil received a patent for the work on August 11, 1942. This technique would later be used again, first in the 1950s at Sylvania Electronic Systems Division and is used in everyday technology such as Bluetooth and Wi-Fi.

Marlyn Wescoff, standing, and Ruth Lichterman reprogram the ENIAC in 1946.
Marlyn Wescoff (standing) and Ruth Lichterman reprogram the ENIAC in 1946
The programmers of the ENIAC computer in 1944, were six female mathematicians; Marlyn Meltzer, Betty Holberton, Kathleen Antonelli, Ruth Teitelbaum, Jean Bartik, and Frances Spence, who were human computers at the Moore School's computation lab.[63] Adele Goldstine was their teacher and trainer and they were known as the "ENIAC girls." The women who worked on ENIAC were warned that they would not be promoted into professional ratings which were only for men.[66] Designing the hardware was "men's work" and programming the software was "women's work."[67] Sometimes women were given blueprints and wiring diagrams to figure out how the machine worked and how to program it. They learned how the ENIAC worked by repairing it, sometimes crawling through the computer, and by fixing "bugs" in the machinery.[68] Even though the programmers were supposed to be doing the "soft" work of programming, in reality, they did that and fully understood and worked with the hardware of the ENIAC. When the ENIAC was revealed in 1946, Goldstine and the other women prepared the machine and the demonstration programs it ran for the public. None of their work in preparing the demonstrations was mentioned in the official accounts of the public events. After the demonstration, the university hosted an expensive celebratory dinner to which none of the ENIAC six were invited.

In Canada, Beatrice Worsley started working at the National Research Council of Canada in 1947 where she was an aerodynamics research officer.[73] A year later, she started working in the new Computational Centre at the University of Toronto.[73] She built a differential analyzer in 1948 and also worked with IBM machines in order to do calculations for Atomic Energy of Canada Limited. She went to study the EDSAC at the University of Cambridge in 1949. She wrote the program that was run the first time EDSAC performed its first calculations on May 6, 1949.

Grace Hopper was the first person to create a compiler for a programming language and one of the first programmers of the Harvard Mark I computer, an electro-mechanical computer based on Analytical Engine. Hopper's work with computers started in 1943, when she started working at the Bureau of Ordnance's Computation Project at Harvard where she programmed the Harvard Mark I.[49] Hopper not only programmed the computer, but created a 500-page comprehensive manual for it.[75] Even though Hopper created the manual, which was widely cited and published, she was not specifically credited in it. Hopper is often credited with the coining of the term "bug" and "debugging" when a moth caused the Mark II to malfunction. While a moth was found and the process of removing it called "debugging," the terms were already part of the language of programmers.

**1950s**

Grace Hopper continued to contribute to computer science through the 1950s. She brought the idea of using compilers from her time at Harvard to UNIVAC which she joined in 1949. Other women who were hired to program UNIVAC included Adele Mildred Koss, Frances E. Holberton, Jean Bartik, Frances Morello and Lillian Jay. To program the UNIVAC, Hopper and her team used the FLOW-MATIC programming language, which she developed. Holberton wrote a code, C-10, that allowed for keyboard inputs into a general-purpose computer. Holberton also developed the Sort-Merge Generator in 1951 which was used on the UNIVAC I. The Sort-Merge Generator marked the first time a computer "used a program to write a program." Holberton suggested that computer housing should be beige or oatmeal in color which became a long-lasting trend. Koss worked with Hopper on various algorithms and a program that was a precursor to a report generator.

Klara Dan von Neumann was one of the main programmers of the MANIAC, a more advanced version of ENIAC. Her work helped the field of meteorology and weather prediction.

The NACA, and subsequently NASA, recruited women computers following World War II.[43] By the 1950s, a team was performing mathematical calculations at the Lewis Research Center in Cleveland, Ohio, including Annie Easley, Katherine Johnson and Kathryn Peddrew. At the National Bureau of Standards, Margaret R. Fox was hired to work as part of the technical staff of the Electronic Computer Laboratory in 1951. In 1956, Gladys West was hired by the U.S. Naval Weapons Laboratory as a human computer. West was involved in calculations that let to the development of GPS.

At Convair Aircraft Corporation, Joyce Currie Little was one of the original programmers for analyzing data received from the wind tunnels. She used punch cards on an IBM 650 which was located in a different building from the wind tunnel. To save time in the physical delivery of the punch cards, she and her colleague, Maggie DeCaro, put on roller skates to get to and from the building faster.

In Israel, Thelma Estrin worked on the design and development of WEIZAC, one of the world's first large-scale programmable electronic computers. In the Soviet Union a team of women helped design and build the first digital computer in 1951. In the UK, Kathleen Booth worked with her husband, Andrew Booth on several computers at Birkbeck College. Kathleen Booth was the programmer and Andrew built the machines. Kathleen developed Assembly Language at this time.

**1960s**

![__](https://upload.wikimedia.org/wikipedia/commons/thumb/d/db/Margaret_Hamilton_-_restoration.jpg/1920px-Margaret_Hamilton_-_restoration.jpg)

### **Margaret Hamilton in 1969, standing next to listings of the software she and her MIT team produced for the Apollo project**

PFC Patricia Barbeau operates a tape-drive on the IBM 729 at Camp Smith.
PFC Patricia Barbeau operates a tape-drive on the IBM 729 at Camp Smith.
Milly Koss who had worked at UNIVAC with Hopper, started work at Control Data Corporation (CDC) in 1965. There she developed algorithms for graphics, including graphic storage and retrieval.

Mary K. Hawes of Burroughs Corporation set up a meeting in 1959 to discuss the creation a computer language that would be shared between businesses. Six people, including Hopper, attended to discuss the philosophy of creating a common business language (CBL). Hopper became involved in developing COBOL (Common Business Oriented Language) where she innovated new symbolic ways to write computer code. Hopper developed programming language that was easier to read and "self-documenting." After COBOL was submitted to the CODASYL Executive Committee, Betty Holberton did further editing on the language before it was submitted to the Government Printing Office in 1960. IBM were slow to adopt COBOL, which hindered its progress but it was accepted as a standard in 1962, after Hopper had demonstrated the compiler working both on UNIVAC and RCA computers. The development of COBOL led to the generation of compilers and generators, most of which were created or refined by women such as Koss, Nora Moser, Deborah Davidson, Sue Knapp, Gertrude Tierney and Jean E. Sammet.

Sammet, who worked at IBM starting in 1961 was responsible for developing the programming language, FORMAC.[96] She published a book, Programming Languages: History and Fundamentals (1969), which was considered the "standard work on programming languages," according to Denise Gürer. It was "one of the most used books in the field," according to The Times in 1972.

Between 1961 and 1963, Margaret Hamilton began to study software reliability while she was working at the US SAGE air defense system. In 1965, she was responsible for programming the software for the onboard flight software on the Apollo mission computers. After Hamilton had completed the program, the code was sent to Raytheon where "expert seamstresses" called the "Little Old Ladies" actually hardwired the code by threading copper wire through magnetic rings. Each system could store more than 12,000 words that were represented by the copper wires.

In 1964, the British Prime Minister Harold Wilson announced a "White-Hot" revolution in technology, that would give greater prominence to IT work. As women still held most computing and programming positions at this time, it was hoped that it would give them more positive career prospects. In 1965, Sister Mary Kenneth Keller became the first American woman to earn a doctorate in computer science. Keller helped develop BASIC while working as a graduate student at Dartmouth, where the university "broke the 'men only' rule" so she could use its computer science center.

In 1966, Frances "Fran" Elizabeth Allen who was developing programming language compilers at IBM Research, published a paper entitled "Program Optimization,". It laid the conceptual basis for systematic analysis and transformation of computer programs. This paper introduced the use of graph-theoretic structures to encode program content in order to automatically and efficiently derive relationships and identify opportunities for optimization.

Christine Darden began working for NASA's computing pool in 1967 having graduated from the Hampton Institute. Women were involved in the development of Whirlwind, including Judy Clapp. She created the prototype for an air defense system for Whirlwind which used radar input to track planes in the air and could direct aircraft courses.

In 1969, Elizabeth "Jake" Feinler, who was working for Stanford, made the first Resource Handbook for ARPANET. This led to the creation of the ARPANET directory, which was built by Feinler with a staff of mostly women. Without the directory, "it was nearly impossible to navigate the ARPANET."

By the end of the decade, the general demographics of programmers had shifted away from being predominantly women, as they had before the 1940s. Though women accounted for around 30 to 50 percent of computer programmers during the 1960s, few were promoted to leadership roles and women were paid significantly less than their male counterparts. Cosmopolitan ran an article in the April 1967 issue about women in programming called "The Computer Girls." Even while magazines such as Cosmopolitan saw a bright future for women in computers and computer programming in the 1960s, the reality was that women were still being marginalized.

**1970s**

In the early 1970s, Pam Hardt-English led a group to create a computer network they named Resource One and which was part of a group called Project One.[117] Her idea to connect Bay Area bookstores, libraries and Project One was an early prototype of the Internet. To work on the project, Hardt-English obtained an expensive SDS-940 computer as a donation from TransAmerica Leasing Corporation in April 1972.They created an electronic library and housed it in a record store called Leopold's in Berkeley. This became the Community Memory database and was maintained by hacker Jude Milhon. After 1975, the SDS-940 computer was repurposed by Sherry Reson, Mya Shone, Chris Macie and Mary Janowitz to create a social services database and a Social Services Referral Directory. Hard copies of the directory, printed out as a subscription service, were kept at city buildings and libraries. The database was maintained and in use until 2009.

In the early 1970s, Elizabeth "Jake" Feinler, who worked on the Resource Directory for ARPANET, and her team created the first WHOIS directory. Feinler set up a server at the Network Information Center (NIC) at Stanford which would work as a directory that could retrieve relevant information about a person or entity. She and her team worked on the creation of domains, with Feinler suggesting that domains be divided by categories based on where the computers were kept. For example, military computers would have the domain of .mil, computers at educational institutions would have .edu. Feinler worked for NIC until 1989.

Adele Goldberg was one of seven programmers that developed Smalltalk in the 1970s, and wrote the majority of the language's documentation. It was one of the first object-oriented programming languages the base of the current graphic user interface, that has its roots in the 1968 The Mother of All Demos by Douglas Engelbart. Smalltalk was used by Apple to launch Apple Lisa in 1983, the first personal computer with a GUI, and a year later its Macintosh. Windows 1.0, based on the same principles, was launched a few months later in 1985.

In the late 1970s, women such as Paulson and Sue Finley wrote programs for the Voyager mission. Voyager continues to carry their codes inside its own memory banks as it leaves the solar system. In 1979, Ruzena Bajcsy founded the General Robotics, Automation, Sensing and Perception (GRASP) Lab at the University of Pennsylvania.

In the mid-70s, Joan Margaret Winters began working at IBM as part of a "human factors project," called SHARE. In 1978, Winters was the deputy manager of the project and went on to lead the project between 1983 and 1987. The SHARE group worked on researching how software should be designed to consider human factors.

**1980s**

In 1981, Deborah Washington Brown became the first African American woman to earn a Ph.D. in computer science from Harvard University (at the time the degree was part of the applied mathematics program). Her thesis was titled "The solution of difference equations describing array manipulation in program loops". Shortly after, in 1982, Marsha R. Williams became the second African American woman to earn a Ph.D. in computer science.

Sometimes known as the "Betsy Ross of the personal computer," according to the New York Times, Susan Kare worked with Steve Jobs to design the original icons for the Macintosh. Kare designed the moving watch, paintbrush and trash can elements that made using a Mac user-friendly. Kare worked for Apple until the mid-1980s, going on to work on icons for Windows 3.0. Other types of computer graphics were being developed by Nadia Magnenat Thalmann in Canada. Thalmann started working on computer animation to develop "realistic virtual actors" first at the University of Montréal in 1980 and later in 1988 at the École Polytechnique Fédérale de Lausanne.

Computer and video games became popular in the 1980s, but many were primarily action-oriented and not designed from a woman's point of view. Stereotypical characters such as the damsel in distress featured prominently and consequently were not inviting towards women. Dona Bailey designed Centipede, where the player shoots insects, as a reaction to such games, later saying "It didn't seem bad to shoot a bug". Carol Shaw, considered to be the first modern female games designer, released a 3D version of tic-tac-toe for the Atari 2600 in 1980.Roberta Williams and her husband Ken, founded Sierra Online and pioneered the graphic adventure game format in Mystery House and the King's Quest series. The games had a friendly graphical user interface and introduced humor and puzzles. Cited as an important game designer, her influence spread from Sierra to other companies such as LucasArts and beyond. Brenda Laurel ported games from arcade versions to the Atari 8-bit computers in the late 1970s and early 1980s. She then went to work for Activision and later wrote the manual for Maniac Mansion.

1984 was the year of Women into Science and Engineering (WISE Campaign). A 1984 report by Ebury Publishing reported that in a typical family, only 5% of mothers and 19% of daughters were using a computer at home, compared to 25% of fathers and 51% of sons. To counteract this, the company launched a series of software titles designed towards women and publicized in Good Housekeeping.[146] Anita Borg, who had been noticing that women were under-represented in computer science, founded an email support group, Systers, in 1987.

As Ethernet became the standard for networking computers locally, Radia Perlman, who worked at Digital Equipment Corporation (DEC), was asked to "fix" limitations that Ethernet imposed on large network traffic. In 1985, Perlman came up with a way to route information packets from one computer to another in an "infinitely scalable" way that allowed large networks like the Internet to function.Her solution took less than a few days to design and write up. The name of the algorithm she created is the Spanning Tree Protocol. In 1986, Lixia Zhang was the only woman and graduate student to participate in the early Internet Engineering Task Force (IETF) meetings. Zhang was involved in early Internet development.

In Europe, project was developed in the mid-1980s to create an academic network in Europe using the Open System Interconnection (OSI) standards. Borka Jerman Blažič, a Yugoslavian computer scientist was invited to work on the project. She was involved in establishing a Yugoslav Research and Academic Network (YUNAC) in 1989 and registered the domain of .yu for the country.

In the field of human–computer interaction (HCI), French computer scientist, Joëlle Coutaz developed the presentation-abstraction-control (PAC) model in 1987.[152][153] She founded the User Interface group at the Laboratorire de Génie Informatique of IMAG where they worked on different problems relating to user interface and other software tools.

In 1988, Stacy Horn, who had been introduced to bulletin board systems (BBS) through The WELL, decided to create her own online community in New York, which she called the East Coast Hang Out (ECHO).Horn invested her own money and pitched the idea for ECHO to others after bankers refused to hear her business plan. Horn built her BBS using UNIX, which she and her friends taught to one another.Eventually ECHO moved an office in Tribeca in the early 1990s and started getting press attention. ECHO's users could post about topics that interested them, and chat with one another, and were provided email accounts. Around half of ECHO's users were women. ECHO was still online as of 2018.

**1990s**

By the 1990s, computing was dominated by men. The proportion of female computer science graduates peaked in 1984 around 37 per cent, and then steadily declined.[162] Although the end of the 20th century saw an increase in women scientists and engineers, this did not hold true for computing, which stagnated.[163] Despite this, they were very involved in working on hypertext and hypermedia projects in the late 1980s and early 1990s. A team of women at Brown University, including Nicole Yankelovich and Karen Catlin, developed Intermedia and invented the anchor link. Apple partially funded their project and incorporated their concepts into Apple operating systems. Sun Microsystems Sun Link Service was developed by Amy Pearl. Janet Walker developed the first system to use bookmarks when she created the Symbolics Document Examiner. In 1989, Wendy Hall created a hypertext project called Microcosm, which was based on digitized multimedia material found in the Mountbatten archive. Cathy Marshall worked on the NoteCards system at Xerox PARC. NoteCards went on to influence Apple's HyperCard. As the Internet became the World Wide Web, developers like Hall adapted their programs to include Web viewers. Her Microcosm was especially adaptable to new technologies, including animation and 3-D models. In 1994, Hall helped organize the first conference for the Web.

Following the increased popularity of the Internet in the 1990s, online spaces were set up to cater for women, including the online community Women's WIRE and the technical and support forum LinuxChix. Women's WIRE, launched by Nancy Rhine and Ellen Pack in October 1993, was the first Internet company to specifically target this demographic. A conference for women in computer-related jobs, the Grace Hopper Celebration of Women in Computing, was first launched in 1994 by Anita Borg.

Game designer Brenda Laurel started working at Interval Research in 1992, and began to think about the differences in the way girls and boys experienced playing video games. After interviewing around 1,000 children and 500 adults, she determined that games weren't designed with girls' interests in mind.[178] The girls she spoke with wanted more games with open worlds and characters they could interact with. Her research led to Interval Research giving Laurel's research team their own company in 1996, Purple Moon. Also in 1996, Mattel's game, Barbie Fashion Designer, became the first best-selling game for girls. Purple Moon's first two games based on a character called Rockett, made it to the 100 best-selling games in the years they were released. In 1999, Mattel bought out Purple Moon.


![__](https://upload.wikimedia.org/wikipedia/commons/4/40/Jaimelevyrussell.jpg)

### **Jaime Levy helped popularise the e-Zine in the 1990s**

Jaime Levy created one of the first e-Zines in the early 1990s, starting with CyberRag, which included articles, games and animations loaded onto diskettes that anyone with a Mac could access. Later, she renamed the zine to Electronic Hollywood.Billy Idol commissioned Levy to create a disk for his album, Cyberpunk. She was hired to be the creative director of the online magazine, Word, in 1995.

Cyberfeminists, VNS Matrix, made up of Josephine Starrs, Juliane Pierce, Francesca da Rimini and Virginia Barratt, created art in the early 1990s linking computer technology and women's bodies. In 1997, there was a gathering of cyberfeminists in Kassel, called the First Cyberfeminist International.

In China, Hu Qiheng, was the leader of the team who installed the first TCP/IP connection for China, connecting to the Internet on April 20, 1994. In 1995, Rosemary Candlin went to write software for CERN in Geneva. In the early 1990s, Nancy Hafkin was an important figure in working with the Association for Progressive Communications (APC) in enabling email connections in 10 African countries. Starting in 1999, Anne-Marie Eklund Löwinder began to work with Domain Name System Security Extensions (DNSSEC) in Sweden.[189] She later made sure that the domain, .se, was the world's first top level domain name to be signed with DNSSEC.

From the late 1980s until the mid-1990s, Misha Mahowald developed several key foundations of the field of Neuromorphic engineering, while working at the California Institute of Technology and later at the ETH Zurich. More than 20 years after her untimely death, the Misha Mahowald Prize was named after her to recognize excellence in the field which she helped to create.

**2000s**

In the 21st century, several attempts have been made to reduce the gender disparity in IT and get more women involved in computing again. A 2001 survey found that while both sexes use computers and the internet in equal measure, women were still five times less likely to choose it as a career or study the subject beyond standard secondary education. Journalist Emily Chang said a key problem has been personality tests in job interviews and the belief that good programmers are introverts, which tends to self-select the stereotype of an asocial white male nerd.

In 2004, the National Center for Women & Information Technology was established by Lucy Sanders to address the gender gap.[194] Carnegie Mellon University has made a concerted attempt to increase gender diversity in the computer science field, by selecting students based on a wide criteria including leadership ability, a sense of "giving back to the community" and high attainment in maths and science, instead of traditional computer programming expertise. As well as increase the intake of women into CMU, the programme produced better quality students because of the increased diversity making a stronger team.

**2010s**

Despite the pioneering work of some designers, video games are still considered biased towards men. A 2013 survey by the International Game Developers Association revealed only 22% of game designers are women, although this is substantially higher than figures in previous decades. Working to bring inclusion to the world of open source project development, Coraline Ada Ehmke drafted the Contributor Covenant in 2014. By 2018, over 40,000 software projects have started using the Contributor Covenant, including TensorFlow, Vue and Linux. In 2014, Danielle George, professor at the School of Electrical and Electronic Engineering, University of Manchester spoke at the Royal Institution Christmas Lectures on the subject of "how to hack your home", describing simple experiments involving computer hardware and demonstrating a giant game of Tetris by remote controlling lights in an office building.

In 2017, Michelle Simmons founded the first quantum computing company in Australia.[199] The team, which has made "great strides" in 2018, plans to develop a 10-qubit prototype silicon quantum integrated circuit by 2022. In the same year, Doina Precup became the head of DeepMind Montreal, working on artificial intelligence. Xaviera Kowo is a programmer from Cameroon, who won the Margaret award, for programming a robot which processes waste in 2022.

**2020s**

In 2023 the EU-Startups the leading online publication with a focus on startups in Europe published the list of top 100 of the most influential women in the startup and venture capital space in Europe.[202] The theme of the list reflects the era of innovation and technological change. That being said, there are plenty of inspiring women in Europe's startup and all around the world in VC space who are making daily changes possible and encouraging a new generation of female for entrepreneurship and innovation.